# FRED API Test

-   2025/8/20 開発途中
-   [GEMINI](https://gemini.google.com/app/1348d8b8b8741164?utm_source=app_launcher&utm_medium=owned&utm_campaign=base_all)
-   CSV 形式でのデータ保存と SQLite 形式でのデータ保存


In [ ]:
import pandas as pd
import polars as pl
from pathlib import Path
import sqlite3
import json
from datetime import timedelta
import time
import os
from dotenv import load_dotenv
from fredapi import Fred
from tqdm import tqdm
import warnings


warnings.simplefilter("ignore")


def find_ancestor_dir(dir_name: str) -> Path | None:
    """
    カレントディレクトリの親ディレクトリを名前を指定して探し出す関数
    """
    current_dir = Path().cwd()
    for parent in current_dir.parents:
        if parent.name == dir_name:
            return parent

    return None


def get_fred_series_incrementally(
    series_id: str, frequency: str, fred_instance: Fred
) -> pd.Series:
    """
    キャッシュの鮮度をチェックし、古い場合は差分データのみをAPIから取得して更新する。

    Args:
        series_id (str): FREDのシリーズID
        frequency (str): データ更新頻度
        fred_instance (Fred): Fred APIのインスタンス

    Returns:
        pd.Series: 時系列データ
    """
    cache_file_path = os.path.join(CACHE_DIR, f"{series_id}.csv")
    freshness_threshold_days = {"日次": 1, "週次": 7, "月次": 30, "四半期": 90}.get(
        frequency, 1
    )

    # 1. キャッシュが存在しない場合は、全データを取得
    if not os.path.exists(cache_file_path):
        print(
            f"📉 キャッシュが存在しないため、全データをAPIから取得します: '{series_id}'"
        )
        try:
            full_data = fred_instance.get_series(series_id)
            full_data.to_csv(cache_file_path)
            return full_data
        except Exception as e:
            print(f"エラー: {series_id} の取得に失敗しました: {e}")
            return None

    # 2. キャッシュが存在する場合、鮮度をチェック
    file_mod_time = os.path.getmtime(cache_file_path)
    age_days = (time.time() - file_mod_time) / (60 * 60 * 24)

    # 2a. キャッシュが新鮮な場合
    if age_days <= freshness_threshold_days:
        print(f"✅ キャッシュは新鮮です。'{series_id}' をファイルから読み込みます。")
        return pd.read_csv(cache_file_path, index_col=0, parse_dates=True).iloc[:, 0]

    # 2b. キャッシュが古い場合 -> 差分更新を実行
    print(f"🔄 キャッシュが古いため、'{series_id}' の差分更新を試みます。")
    try:
        # 既存データを読み込み、最後の日付を取得
        existing_data = pd.read_csv(
            cache_file_path, index_col=0, parse_dates=True
        ).iloc[:, 0]
        last_date = existing_data.index.max()

        # 取得開始日を最後の日付の翌日に設定
        start_date_for_new_data = last_date + timedelta(days=1)

        # APIから新しいデータだけを取得
        print(
            f"   -> {start_date_for_new_data.strftime('%Y-%m-%d')}以降のデータをAPIにリクエストします。"
        )
        new_data = fred_instance.get_series(
            series_id, observation_start=start_date_for_new_data
        )

        # 新しいデータがあるかチェック
        if not new_data.empty:
            # 既存データと新データを結合
            combined_data = pd.concat([existing_data, new_data])
            # 重複（もしあれば）を削除
            combined_data = combined_data[~combined_data.index.duplicated(keep="last")]

            # キャッシュを更新
            combined_data.to_csv(cache_file_path)
            print(f"   -> {len(new_data)}件の新規データを追加しました。")
            return combined_data
        else:
            # 新しいデータがない場合、ファイルの更新時刻だけを更新して「チェック済み」とする
            print("   -> 新規データはありませんでした。")
            os.utime(cache_file_path, None)  # タイムスタンプを現在時刻に更新
            return existing_data

    except Exception as e:
        print(f"エラー: {series_id} の差分更新に失敗しました: {e}")
        # エラー発生時は既存のデータを返す
        return pd.read_csv(cache_file_path, index_col=0, parse_dates=True).iloc[:, 0]


ROOT_DIR = find_ancestor_dir("papers-1")
Q_DIR = Path().cwd().parent.parent
DATA_DIR = Q_DIR / "data" / "MSCI_KOKUSAI"
PRJ_DIR = Q_DIR / "A_001"
ROOT_DIR = find_ancestor_dir("papers-1")
FRED_DIR = ROOT_DIR / "FRED"
CACHE_DIR = FRED_DIR / "cache"


# FRED
load_dotenv()
api_fred = os.getenv("FRED_API_KEY")

with open(FRED_DIR / "fred_series.json", "r", encoding="utf-8") as f:
    series_info = json.load(f)

display(series_info["Interest Rates"])


{'DFF': {'name': 'FF金利 (実効金利)',
  'frequency': '日次',
  'description': '最も基本的な米国の政策金利。金融機関が相互に資金を貸し借りする際の金利（無担保翌日物）の実績値。'},
 'FEDFUNDS': {'name': 'FF金利 (月次平均)',
  'frequency': '月次',
  'description': 'DFFの月次平均値。月次の分析に利用。'},
 'SOFR': {'name': 'SOFR',
  'frequency': '日次',
  'description': '担保付翌日物調達金利。米国債を担保とする翌日物のレポ取引金利。'},
 'DPRIME': {'name': 'プライムレート',
  'frequency': '日次',
  'description': '銀行が最優良企業へ短期融資を行う際の最優遇貸出金利。'},
 'DGS3MO': {'name': '米国債3ヶ月物',
  'frequency': '日次',
  'description': '短期金利の代表格。金融政策の変更に敏感に反応。'},
 'DGS2': {'name': '米国債2年物',
  'frequency': '日次',
  'description': '短期金利。近い将来の金融政策に対する市場の期待を反映。'},
 'DGS10': {'name': '米国債10年物利回り',
  'frequency': '日次',
  'description': '長期金利の最も重要な指標。株価の評価（ディスカウントレート）に影響。'},
 'DGS30': {'name': '米国債30年物',
  'frequency': '日次',
  'description': '超長期の金利。年金基金などの長期投資家の動向を反映。'},
 'T10Y2Y': {'name': '長短金利差 (10年-2年)',
  'frequency': '日次',
  'description': '景気後退の強力な先行指標。マイナス（逆イールド）は景気後退を示唆。'},
 'T10Y3M': {'name': '長短金利差 (10年-3ヶ月)',
  'frequency': '日次',
